In [1]:
import pickle
import os
import pandas as pd
from matplotlib import pyplot as plt
os.chdir(os.path.expanduser('~/wcEcoli/'))
# noinspection PyUnresolvedReferences
from wholecell.io.tablereader import TableReader
import io
import numpy as np
from wholecell.io import tsv
from wholecell.utils.filepath import ROOT_PATH
import plotly.graph_objects as go
from models.ecoli.analysis import cohortAnalysisPlot
from wholecell.analysis.analysis_tools import (exportFigure,
	read_bulk_molecule_counts, read_stacked_bulk_molecules, read_stacked_columns)
from wholecell.io.tablereader import TableReader
from sklearn.metrics import r2_score
import plotly.graph_objects as go
import glob
import plotly.express as px

In [2]:
# function to match gene symbols to monomer ids
def get_gene_symbols_for_monomer_ids():
	"""
	Extracts the gene symbols for each monomer id in the model.
	Returns: a dictionary mapping monomer ids to gene symbols.
	Code adapted from convert_to_flat.py.
	"""
	RNAS_FILE = os.path.join(ROOT_PATH, 'reconstruction', 'ecoli',
								 'flat', 'rnas.tsv')
	with (io.open(RNAS_FILE, 'rb') as f):
		reader = tsv.reader(f, delimiter='\t')
		headers = next(reader)
		while headers[0].startswith('#'):
			headers = next(reader)

		# extract relevant information
		gene_symbol_index = headers.index('common_name')
		protein_id_index = headers.index('monomer_ids')
		monomer_ids_to_gene_symbols = {}
		for line in reader:
			gene_symbol = line[gene_symbol_index]
			protein_id = list(
				line[protein_id_index][2:-2].split('", "'))[0]
			monomer_ids_to_gene_symbols[protein_id] = gene_symbol

	return monomer_ids_to_gene_symbols

def get_common_name(protein_id):
    """
    Get the common name of a protein given its monomer id.
    Args:
        protein_id: the name of the monomer

    Returns:
        common_name: The common name of the protein.

    """
    if '[' in protein_id:
        protein_id = protein_id[:-3]  # subtract the compartment
        
    if protein_id == 'NG-GFP-MONOMER':
        print('NEW GENE PRESENT')
    else:
        common_name = get_gene_symbols_for_monomer_ids()[protein_id]
    return common_name

In [ ]:
current_sequence = "CLClim3NE3" # y axis
CLNE_sequence = "CLNE3"

In [ ]:
# edit the plot out file as needed
out_pth = f"out/figures/PDR_UPDATE_MERGE/F2/change_in_half_life_histograms/counts_plot__HL_bounds[{HL_lower},{HL_middle},{HL_upper}]__PC_bounds[{PC_lower},{PC_middle},{PC_upper}]__{current_sequence}_{CLNE_sequence}.png"

column = 'HL_log2_fold_change'
validaiton_column = 'Log10 Validation Data Average Monomer Counts'

CLClim3NE1_HLs_Clim3_above_upper, CLClim3NE1_HLs_Clim3_upper_to_zero, CLClim3NE1_HLs_Clim3_middle, CLClim3NE1_HLs_Clim3_0_to_lower, CLClim3NE1_HLs_Clim3_lower, words_u, words_u2m, words_m, words_m2l, words_l = obtain_data(df, column, HL_upper, HL_middle, HL_lower)


# ok now make a 2 by 2 plot

def make_counts_comparison_with_upper_HL_change_data(row, col, whole_df, interest_df):
    # define the axes:
    ax = axes[row, col]
    
    # plot all of the proteins 
    sns.scatterplot(data=whole_df, x=whole_df[CLNE_new_name], y=whole_df[CLClimNE_new_name], color='grey', size=0.1, alpha=0.1, ax=ax, )
    
    # plot the proteins of interest:
    sns.scatterplot(data=interest_df, x=interest_df[CLNE_new_name], y=interest_df[CLClimNE_new_name], color='red', alpha=0.5, ax=ax, )
    ax.plot([0, 6], [0, 6], color='black', linestyle='--', linewidth=.5, alpha=0.5)
    
    # plot the common name of the proteins of interest:
    for i, name in enumerate(interest_df['common_name'].unique()):
        # get the index of the protein:
        index = interest_df[interest_df['common_name'] == name].index[0]
        # get the x and y values:
        x = interest_df[CLNE_new_name][index]
        y = interest_df[CLClimNE_new_name][index]
        # plot the common name:
        ax.text(x, y, name, ha='center', va='bottom', fontsize=8, rotation=0, )
    
    # Plot specs:
    ax.set_title(f'Comparison of the log10 average protein counts between the\n2025 and 2020 model with proteins having the highest\nlog2 fold $\\Delta$ in half life highlighted (n={whole_df.shape[0]})', fontsize=10, pad=15, )
    ax.set_xlabel(f'$Log10$(average protein counts +1),\n2020 model ({CLNE_sequence})', fontsize=8, color="black", )
    ax.set_ylabel(f'$Log_10$(average protein counts +1),\n2025 model ({current_sequence})', fontsize=8, color="black", )
    
    # add a legend:
    #legend_labels = [f'all proteins (n={whole_df.shape[0]})', f'proteins with the largest +$\\Delta$\n in half life (n={interest_df.shape[0]})']
    #handles, _ = ax.get_legend_handles_labels()
    #ax.legend(handles=handles, labels=legend_labels, loc='upper left', fontsize=8, frameon=False, markerscale=2)
    ax.legend()
    ax.get_legend().set_visible(False)
 
    

    
def make_counts_comparison_with_lower_HL_change_data(row, col, whole_df, interest_df):
    # define the axes:
    ax = axes[row, col]
    
    # plot all of the proteins 
    sns.scatterplot(data=whole_df, x=whole_df[CLNE_new_name], y=whole_df[CLClimNE_new_name], color='grey', size=.1, alpha=0.1, ax=ax)
    
    # plot the proteins of interest:
    sns.scatterplot(data=interest_df, x=interest_df[CLNE_new_name], y=interest_df[CLClimNE_new_name], color='blue', alpha=0.5, ax=ax)
    ax.plot([0, 6], [0, 6], color='black', linestyle='--', linewidth=.5, alpha=0.5)

    
    # plot the common name of the proteins of interest:
    for i, name in enumerate(interest_df['common_name'].unique()):
        # get the index of the protein:
        index = interest_df[interest_df['common_name'] == name].index[0]
        # get the x and y values:
        x = interest_df[CLNE_new_name][index]
        y = interest_df[CLClimNE_new_name][index]
        # plot the common name:
        ax.text(x, y, name, ha='center', va='bottom', fontsize=8, rotation=0, )
    
    # Plot specs:
    ax.set_title(f'Comparison of the log10 average protein counts between the\n2025 and 2020 model with proteins having the smallest\nlog2 fold $\\Delta$ in half life highlighted (n={whole_df.shape[0]})', fontsize=10, pad=15, )
    ax.set_xlabel(f'$Log10$(average protein counts +1),\n2020 model ({CLNE_sequence})', fontsize=8, color="black", )
    ax.set_ylabel(f'$Log_10$(average protein counts +1),\n2025 model ({current_sequence})', fontsize=8, color="black", )
    ax.legend()
    ax.get_legend().set_visible(False)
    
# now make a plot for the validation comparison: 
def make_counts_comparison_with_upper_HL_change_validation_data(row, col, whole_df, interest_df):
    # define the axes:
    ax = axes[row, col]
    
    # clean up the data: 
    x_col = 'Log10 Validation Data Average Monomer Counts'
    y_col = CLClimNE_new_name
    y_col2 = CLNE_new_name

    # remove proteins that have "None" in their y_col: 
    whole_df_to_plot = whole_df[whole_df[x_col].notna()]
    interest_df_to_plot = interest_df[interest_df[x_col].notna()]

    
    # plot all of the proteins 
    sns.scatterplot(data=whole_df_to_plot, x=whole_df_to_plot[x_col], y=whole_df_to_plot[y_col], color='grey', size=.1, alpha=0.1, ax=ax,)
    
    # plot the proteins of interest:
    sns.scatterplot(data=interest_df_to_plot, x=interest_df_to_plot[x_col], y=interest_df_to_plot[y_col], color='red', alpha=0.9, ax=ax)
    sns.scatterplot(data=interest_df_to_plot, x=interest_df_to_plot[x_col], y=interest_df_to_plot[y_col2], color='black', alpha=0.5, ax=ax)
    
    # add a y=x dotted line:
    ax.plot([0, 6], [0, 6], color='black', linestyle='--', linewidth=.5, alpha=0.5)
    
    # plot the common name of the proteins of interest:
    for i, name in enumerate(interest_df_to_plot['common_name'].unique()):
        # get the index of the protein:
        index = interest_df_to_plot[interest_df_to_plot['common_name'] == name].index[0]
        # get the x and y values:
        x = interest_df_to_plot[x_col][index]
        y = interest_df_to_plot[y_col][index]
        # plot the common name:
        ax.text(x+.2, y, name, ha='center', va='bottom', fontsize=8, rotation=0, )
        
        # also make a line between the two: 
        y2 = interest_df_to_plot[y_col2][index]
        difference = y - y2
        if difference > 0.2:
            dx = x-x
            dy = y - y2
            #ax.arrow([x, x], [y, y2], color='red', linewidth=.5, alpha=0.5)
            ax.arrow(x, y2, dx, dy, color='red', linewidth=.5, alpha=0.5, head_width=0.1, head_length=0.15, length_includes_head=True)
    
    # Plot specs:
    ax.set_title(f'Comparison of the log10 average protein counts between the\n2025 model and validation data with proteins having the highest\nlog2 fold $\\Delta$ in half life highlighted (n={whole_df_to_plot.shape[0]})', fontsize=10, pad=15, )
    ax.set_xlabel(f'$Log10$(average protein counts +1),\nValidation Data (Schmidt et al. 2016)', fontsize=8, color="black", )
    ax.set_ylabel(f'$Log_10$(average protein counts +1),\n2025 model ({current_sequence})', fontsize=8, color="black", )
    ax.legend()
    ax.get_legend().set_visible(False)


# now make a plot for the validation comparison: 
def make_counts_comparison_with_lower_HL_change_validation_data(row, col, whole_df, interest_df):
    # define the axes:
    ax = axes[row, col]
    
    # clean up the data: 
    x_col = 'Log10 Validation Data Average Monomer Counts'
    y_col = CLClimNE_new_name
    y_col2 = CLNE_new_name

    # remove proteins that have "None" in their y_col: 
    whole_df_to_plot = whole_df[whole_df[x_col].notna()]
    interest_df_to_plot = interest_df[interest_df[x_col].notna()]

    
    # plot all of the proteins 
    sns.scatterplot(data=whole_df_to_plot, x=whole_df_to_plot[x_col], y=whole_df_to_plot[y_col], color='grey', size=.1, alpha=0.1, ax=ax,)
    
    # plot the proteins of interest:
    sns.scatterplot(data=interest_df_to_plot, x=interest_df_to_plot[x_col], y=interest_df_to_plot[y_col], color='blue', alpha=0.9, ax=ax, )
    sns.scatterplot(data=interest_df_to_plot, x=interest_df_to_plot[x_col], y=interest_df_to_plot[y_col2], color='black', alpha=0.5, ax=ax, 
                   )
    
    # add a y=x dotted line:
    ax.plot([0, 6], [0, 6], color='black', linestyle='--', linewidth=.5, alpha=0.5)
    
    # plot the common name of the proteins of interest:
    for i, name in enumerate(interest_df_to_plot['common_name'].unique()):
        # get the index of the protein:
        index = interest_df_to_plot[interest_df_to_plot['common_name'] == name].index[0]
        # get the x and y values:
        x = interest_df_to_plot[x_col][index]
        y = interest_df_to_plot[y_col][index]
        # plot the common name:
        ax.text(x+.2, y, name, ha='center', va='bottom', fontsize=8, rotation=0, )
        
        # also make a line between the two: 
        y2 = interest_df_to_plot[y_col2][index]
        difference = y - y2
        if difference < -0.2:
            dx = x-x
            dy = y - y2
            #ax.arrow([x, x], [y, y2], color='red', linewidth=.5, alpha=0.5)
            ax.arrow(x, y2, dx, dy, color='blue', linewidth=.5, alpha=0.5, head_width=0.1, head_length=0.15, length_includes_head=True)
    
    # Plot specs:
    ax.set_title(f'Comparison of the log10 average protein counts between the\n2025 model and validation data with proteins having the smallest\nlog2 fold $\\Delta$ in half life highlighted (n={whole_df_to_plot.shape[0]})', fontsize=10, pad=15, )
    ax.set_xlabel(f'$Log10$(average protein counts +1),\nValidation Data (Schmidt et al. 2016)', fontsize=8, color="black", )
    ax.set_ylabel(f'$Log_10$(average protein counts +1),\n2025 model ({current_sequence})', fontsize=8, color="black", )
    legend_labels = [f"proteins with the largest -$\\Delta$\nin half life in the 2025 model (n={interest_df_to_plot.shape[0]})",  f"2020 model value"]
    ax.legend()
    ax.get_legend().set_visible(False)


# MAKE THE PLOT
# using this: https://sharkcoder.com/data-visualization/mpl-bidirectional
font_color = '#525252'
hfont = {'fontname':'Calibri'}
color_red = '#fd625e'
color_blue = '#01b8aa'
fig, axes = plt.subplots(figsize=(10,10),  nrows=2, ncols=2, gridspec_kw={'height_ratios': [1, 1], 'width_ratios': [1, 1]})
fig.tight_layout()

# plot the plots 
make_counts_comparison_with_upper_HL_change_data(0, 0, combined_df, CLClim3NE1_HLs_Clim3_above_upper)
make_counts_comparison_with_lower_HL_change_data(0, 1, combined_df, CLClim3NE1_HLs_Clim3_lower)
make_counts_comparison_with_upper_HL_change_validation_data(1, 0, combined_df, CLClim3NE1_HLs_Clim3_above_upper)
make_counts_comparison_with_lower_HL_change_validation_data(1, 1, combined_df, CLClim3NE1_HLs_Clim3_lower)

fig.subplots_adjust(wspace=0.3, hspace=0.4)

# save the figure:
out_pth = os.path.expanduser(out_pth)
output_dir = os.path.dirname(out_pth)
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it does not exist
plt.savefig(out_pth, dpi=300, bbox_inches='tight')
